In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings(action = 'ignore')
import pickle
import pandas as pd

In [3]:
#magnetization per spin을 구합니다.
def magnetization(lattice):
    L = len(lattice)
    m = 0
    for i in range(L): 
        for j in range(L): 
            m += lattice[i,j]
    return abs(m)/L**2

def auto_correlation(lattice_size, temperature, equilibrium, number_of_samples):
    config=np.ones((lattice_size,lattice_size));
    correlation,mags=[],[];
    #Until the configuration reaches equilibrium state, repeats MCS.
    for i in range(equilibrium): 
        for j in range(lattice_size**2): 
            x,y=np.random.randint(0,lattice_size),np.random.randint(0,lattice_size)
            s=config[x,y]
            neigh=config[(x+1)%lattice_size,y]+config[x,(y+1)%lattice_size]+config[(x-1)%lattice_size,y]+config[x,(y-1)%lattice_size]
            DE=2*s*neigh
            if DE<0: s*=-1
            elif np.random.rand()<np.exp(-DE*1/temperature):s*=-1
            config[x,y]=s   
    mags.append(magnetization(config))
    
    #now configuration reaches at equilibrium
    #we should sample samples.
    for i in range(number_of_samples-1): #number_of_samples개 만큼의 sample에 대해서 correlation을 살펴본다.  
        for j in range(lattice_size**2):
            x,y=np.random.randint(0,lattice_size),np.random.randint(0,lattice_size)
            s=config[x,y]
            neigh=config[(x+1)%lattice_size,y]+config[x,(y+1)%lattice_size]+config[(x-1)%lattice_size,y]+config[x,(y-1)%lattice_size]
            DE=2*s*neigh
            if DE<0: s*=-1
            elif np.random.rand()<np.exp(-DE*1/temperature):s*=-1
            config[x,y]=s
        mags.append(magnetization(config))    
    #이 부분이 autocorrelation을 계산하는 부분입니다. 
    for t in range(len(mags)):
        mean1,mean2,mean3=0,0,0;
        for j in range(len(mags)-t):
            mean1+=1/(len(mags)-t)*mags[t+j]*mags[j];
            mean2+=1/(len(mags)-t)*mags[t+j];
            mean3+=1/(len(mags)-t)*mags[j];
        correlation.append(mean1-mean2*mean3) #correlation is list of floats
    return correlation

In [5]:
Ens=[]
for tt in tqdm(Temperature,desc="temperature"):
    cor=[]
    x=0
    for ens in range(512):
        cor.append(np.array(auto_correlation(lattice_size=3,temperature=tt,equilibrium=300,number_of_samples=100)))
    for i in range(len(cor)):
        x+=cor[i]/len(cor) #it might be numpy array
    Ens.append(x) #list of array for each temperature

temperature: 100%|██████████| 101/101 [19:59<00:00, 11.88s/it]
